In [1]:
import cv2
import pydub
import pandas as pd
import numpy as np
import librosa as lib
from scipy.io import wavfile as wav
from scipy.signal import stft
import IPython
import soundfile as sf


C:\Users\alpha\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Question: 1

In this question, you are tasked with enhancing the resolution of a video. The goal is to improve the quality of individual frames. You are expected to use basic algorithms for achieving this goal.

### Task 1: Frame Extraction

Extract frames from the video using OpenCV.

### Task 2: Resolution Enhancement

Apply the following enhancement algorithms to scale the extracted frames by a factor of 2:

1) Nearest-neighbor Interpolation <br>
2) Bilinear Interpolation <br>
3) Bicubic Interpolation <br>

Explore these approaches by your self. These are just builtin parameters in resize function.
https://theailearner.com/2018/11/15/image-interpolation-using-opencv-python/

### Task 3: Video Reconstruction

After enhancing the frames, reconstruct the video by merging the enhanced frames while ensuring that the frame rate of the reconstructed video matches that of the original video. Generate a separate video for each interpolation method.

<b>Bonus</b>: Apply a self-selected algorithm to improve video quality.

In [ ]:
vid = cv2.VideoCapture('Q1.mp4')
fps = int(vid.get(cv2.CAP_PROP_FPS))
heightframe = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
widthframe = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
vid_type = cv2.VideoWriter_fourcc(*'mp4v')
enhance_file = cv2.VideoWriter('intercubic.mp4', vid_type, 30, (widthframe * 2, heightframe * 2))

while True:
    ret, frame = vid.read()
    if ret:
        outframe = cv2.resize(frame, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        enhance_file.write(outframe)
        cv2.imshow('Frame', outframe)
        if cv2.waitKey(30) & 0xFF == 27:
            break
    else:
        break
vid.release()
enhance_file.release()
cv2.destroyAllWindows()

In [ ]:
# orgframearr

In [ ]:

vid = cv2.VideoCapture('Q1.mp4')
vid_type = cv2.VideoWriter_fourcc(*'XVID')
enhance_file = cv2.VideoWriter('near_neigh.mp4',vid_type,30,(widthframe*2,heightframe*2))
while True:
    ret,frame = vid.read()
    if ret == True:
        outframe = cv2.resize(frame,None, fx = 2, fy = 2, interpolation = cv2.INTER_NEAREST)
        enhance_file.write(outframe)
    else:
        break
    if cv2.waitKey(30) & 0xFF == 27:
        break
vid.release()
enhance_file.release()
cv2.destroyAllWindows()

In [ ]:

vid = cv2.VideoCapture('Q1.mp4')
vid_type = cv2.VideoWriter_fourcc(*'XVID')
enhance_file = cv2.VideoWriter('linear.mp4',vid_type,29,(widthframe*2,heightframe*2))
while True:
    ret,frame = vid.read()
    if ret == True:
        b = cv2.resize(frame,None, fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)
        enhance_file.write(b)
    else:
        break
    if cv2.waitKey(30) & 0xFF == 27:
        break
vid.release()
enhance_file.release()
cv2.destroyAllWindows()

In [ ]:
#### I dont know name of this exact algo but this looked like it produced better results than others

vid = cv2.VideoCapture('Q1.mp4')
vid_type = cv2.VideoWriter_fourcc(*'XVID')
enhance_file = cv2.VideoWriter('AREA.mp4',vid_type,29,(widthframe*10,heightframe*10))
while True:
    ret,frame = vid.read()
    if ret == True:
        b = cv2.resize(frame,None, fx = 10, fy = 10, interpolation = cv2.INTER_LANCZOS4)  ## this one
        enhance_file.write(b)
    else:
        break
    if cv2.waitKey(30) & 0xFF == 27:
        break
vid.release()
enhance_file.release()
cv2.destroyAllWindows()

##

# Question: 2

In this question, you are tasked with enhancing the audio quality of the video. Follow the given procedure to increase audio quality.

### Step 1: Short-Time Fourier Transform (STFT)
Compute the Short-Time Fourier Transform (STFT) of the audio signal. This operation transforms the audio into the frequency domain over short time intervals.

### Step 2: Magnitude and Phase Extraction
From the STFT, get the magnitude and phase using the np.abs() and np.angle() functions.

### Step 3: Noise Profile Creation
Load the noisy audio and calculate its STFT and magnitude from the STFT. Afterward, compute the average magnitude of the audio along axis=1 to generate a noise profile. This profile is essential for identifying and removing noise.

### Step 4: Adjusting with a Hyperparameter
Multiply the noise profile array by a hyperparameter represented as alpha. Experiment with various values of alpha to fine-tune the results. A good starting point is to set alpha to 2.

### Step 5: Audio Denoising
Subtract the mean noise array from the original audio (You may need to adjust the dimensions of the mean noise array to match with original audio). Ensure that any negative values in the resulting array are replaced with 0. This step effectively reduces noise in the audio.

### Step 6: Incorporating Phase Information
Multiply the modified audio by the complex exponential of the phase information obtained in step 3, which can be expressed as np.exp(1.0j * phase).

### Step 7: Inverse Short-Time Fourier Transform (ISTFT)
Reconstruct the audio by performing the Inverse Short Time Fourier Transform (ISTFT) on the modified audio signal using librosa. Save the resulting audio file.

In [ ]:
ndata , nsr = lib.load("Q2-Noise.wav")
odata, osr  = lib.load('Q2.wav')

In [ ]:

Nst  = lib.stft(ndata)
Ost = lib.stft(odata)
omag = np.abs(Ost)
nmag = np.abs(Nst)                   ## absolute values of
oang  =np.angle(Ost)             # original audio angle
nang = np.angle(Nst)        # noise angle
# print(nmag)
# mean mag
m_mag = np.mean(nmag,1)
hyperparam = m_mag* 4
# print(hyperparam.shape)
# print(omag.shape)
# print(oang.shape)

In [ ]:
# omag = omag.reshape(587,1025)
# omag.shape            ### this comes with some noise

In [ ]:
trans = omag.T     # taking transpose so shapes are equal when subtracting
# print(trans.shape)      # 587
dnoise = []
for i in range(len(trans)):
    t = trans[i] - hyperparam      ## as hyper param is already in a simgle dimension we will iterate over trans to minus it from magnitude of orignial file
    dnoise.append(t)
dnoise = np.array(dnoise)
print(dnoise.shape)
dnoise = dnoise.T
dnoise[dnoise<0] = 0
# dnoise

(587, 1025)
(587, 1025)


In [ ]:
# oang = oang.reshape(587,1025)

In [ ]:
phase = dnoise*np.exp(1.0j*oang)

numpy.ndarray

In [ ]:
mod = lib.istft(phase)
sf.write('dnoisefile.wav',mod,osr)

# Question: 3

For this task, use whisper inference to generate text from the audio file. Use any translation library to translate the text into another language, and then utilize a TTS system to produce audio from the translated text. Supported Languages are :English, Urdu, Arabic

In [2]:
# !pip install tortoise
!git clone https://github.com/152334H/tortoise-tts-fast
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg
!pip install transformers==4.29.2
%cd tortoise-tts-fast
!pip3 install -r requirements.txt --no-deps
!pip3 install -e .
!pip3 install git+https://github.com/152334H/BigVGAN.git

Cloning into 'tortoise-tts-fast'...
remote: Enumerating objects: 2314, done.
remote: Total 2314 (delta 0), reused 0 (delta 0), pack-reused 2314
Receiving objects: 100% (2314/2314), 105.07 MiB | 24.63 MiB/s, done.
Resolving deltas: 100% (1070/1070), done.
Updating files: 100% (697/697), done.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-pqszr0ws
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-pqszr0ws
  Resolved https://github.com/openai/whisper.git to commit fcfeaf1b61994c071bba62da47d7846933576ac9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu j

  Cloning https://github.com/152334H/BigVGAN.git to /tmp/pip-req-build-mj0hrhn1
  Running command git clone --filter=blob:none --quiet https://github.com/152334H/BigVGAN.git /tmp/pip-req-build-mj0hrhn1
  Resolved https://github.com/152334H/BigVGAN.git to commit b10aaeea6ff7e5704d4f8198071cf9a4c174e78a
  Preparing metadata (setup.py) ... done


In [3]:
!pip install whisper


     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.8 kB 660.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7

In [8]:
import whisper

# !pip install ffmpeg-python
# !pip install git+https://github.com/openai/whisper.git
# # import tortoise
# import IPython
# # import torchaudio
# !pip install whisper

TypeError: argument of type 'NoneType' is not iterable

In [4]:
model = load_model("base")
res = model.transcribe('/content/Q3.wav')
audtotext = res['text']

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [5]:
audtotext

' For this task, use whisper inference to generate text from the audio file. Use any translation library to translate the text into another language, and then utilize a TTS system to produce audio from the translated text.'

In [6]:
# !pip install googletrans==3.1.0a0


In [7]:
from googletrans import Translator


In [8]:
mod = Translator()
texttrans = mod.translate(audtotext,'ur')
print(texttrans.text)
urdu_text = texttrans.text

اس کام کے لیے، آڈیو فائل سے متن بنانے کے لیے وسپر انفرنس کا استعمال کریں۔ متن کا دوسری زبان میں ترجمہ کرنے کے لیے کسی بھی ترجمہ لائبریری کا استعمال کریں، اور پھر ترجمہ شدہ متن سے آڈیو بنانے کے لیے TTS سسٹم کا استعمال کریں۔


In [15]:
import IPython
import torchaudio
from tortoise.api import TextToSpeech
# import gTTS

In [16]:
# !pip install
from gtts import gTTS

In [24]:
tts = gTTS(urdu_text, lang = "ur",slow=False)
tts.save('audfileurdu.mp3')

In [11]:
preset = "fast"

In [1]:
# gen = tts.tts_with_preset(urdu_text, voice_samples=None, conditioning_latents=None, preset=preset)
# torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
# IPython.display.Audio('generated.wav')